# 和 Chroma@llamaindex 比较

## 准备

In [1]:
%%time
%%capture

!pip install langchain-chroma
!pip install langchain
!pip install langchain-community

CPU times: user 35.4 ms, sys: 7.09 ms, total: 42.5 ms
Wall time: 10.3 s


## 文档

In [2]:
%%time

books = [
    {
        "name": "围城",
        "description": "主人公方鸿渐留学回国后，面临找工作和个人感情的种种问题。他经历了几段感情波折，包括与鲍小姐的失败婚姻和与孙柔嘉的恋情，最终与孙柔嘉结婚。但婚后生活并不如意，他在事业上也遭遇挫折，未能实现自己的理想。",
        "author": "钱钟书",
    },
    {
        "name": "故乡",
        "description": "小说讲述了主人公“我”（即鲁迅的化身）在阔别多年后回到故乡接母亲到城里生活的故事。在故乡，他遇到了童年的玩伴闰土和老仆人杨二嫂。通过与他们的交谈和观察，主人公感受到故乡的变化和人们生活的困苦。",
        "author": "鲁迅",
    },
    {
       "name": "阿Q正传",
        "description": "讲述了阿Q这个贫苦农民在中国封建社会中的悲惨生活。他虽然穷困潦倒，但心态自负，总是以精神胜利法来安慰自己，逃避现实的困境。然而，随着社会动荡和革命的到来，阿Q的命运变得更加悲惨，最终被误认为是革命党人而被处死。",
        "author": "鲁迅",
    }, 
]

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.58 µs


In [3]:
%%time

from langchain_core.documents import Document

documents=[]

for book in books:
    document=Document(
        page_content=book['description'],
        metadata={"name": book['name'], "author": book['author']},
    )
    documents.append(document)

CPU times: user 122 ms, sys: 11.4 ms, total: 133 ms
Wall time: 133 ms


In [4]:
%%time

from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

CPU times: user 4.81 ms, sys: 0 ns, total: 4.81 ms
Wall time: 4.28 ms


## 嵌入模型

In [5]:
%%time

# from langchain_community.embeddings import OllamaEmbeddings

# ollama_emb = OllamaEmbeddings(
#     model="quentinz/bge-large-zh-v1.5",
#     base_url="http://ape:11435",
# )

# embedding_function =ollama_emb

from langchain.embeddings import HuggingFaceEmbeddings

embedding_function = HuggingFaceEmbeddings(
    model_name="/models/bge-small-zh-v1.5"
)

CPU times: user 2.21 s, sys: 535 ms, total: 2.74 s
Wall time: 2.37 s


## 获取余弦相似度

In [6]:
%%time

from langchain_chroma import Chroma

# load it into Chroma
db = Chroma.from_documents(
    docs, 
    embedding_function,
    collection_metadata={"hnsw:space": "cosine"}
)

CPU times: user 813 ms, sys: 47.1 ms, total: 861 ms
Wall time: 903 ms


In [7]:
%%time

docs = db.similarity_search("方鸿渐")
docs[0].page_content

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


CPU times: user 8.4 ms, sys: 9.72 ms, total: 18.1 ms
Wall time: 17.1 ms


'主人公方鸿渐留学回国后，面临找工作和个人感情的种种问题。他经历了几段感情波折，包括与鲍小姐的失败婚姻和与孙柔嘉的恋情，最终与孙柔嘉结婚。但婚后生活并不如意，他在事业上也遭遇挫折，未能实现自己的理想。'

In [8]:
%%time

docs = db.similarity_search_with_score("方鸿渐")
docs[0]

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 7.28 ms


(Document(page_content='主人公方鸿渐留学回国后，面临找工作和个人感情的种种问题。他经历了几段感情波折，包括与鲍小姐的失败婚姻和与孙柔嘉的恋情，最终与孙柔嘉结婚。但婚后生活并不如意，他在事业上也遭遇挫折，未能实现自己的理想。', metadata={'author': '钱钟书', 'name': '围城'}),
 0.39040178060531616)

In [9]:
1-docs[0][1]

0.6095982193946838